In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForCausalLM
import torch
import json

from ni import HFNIEstimator

/home/tiagoalmeida/safe_volume/A1-datasets/venv-310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_jsonl(file_path):
    def gen():
        with open(file_path) as f:
            for data in map(json.loads, f):
                yield data
                
    return gen

In [15]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", 
                                          padding_side="left")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if tokenizer.eos_token is None and tokenizer.pad_token is None:
    raise RuntimeError("No avaialble padding token")

dataset = Dataset.from_generator(read_jsonl("../../KNOWLAM/clean_datasets/politics__speeches_clean.jsonl"))
#dataset = Dataset.from_generator(read_jsonl("../../KNOWLAM/clean_datasets/math__openai_grade_school_clean.jsonl"))
print(len(dataset))
#dataset = Dataset.from_generator(read_jsonl("../../KNOWLAM/clean_datasets/math__openai_grade_school_clean.jsonl"))
#dataset = dataset.map(lambda sample: tokenizer(sample["text"]),
#                      remove_columns=["text"])

1052


In [18]:
from dataclasses import dataclass
from typing import List

@dataclass
class LMInput:
    input_ids: List[int]
    attention_mask: List[int]
    
    def __len__(self):
        return len(self.input_ids)

class MovingWindow:
    "https://stackoverflow.com/questions/64118654/best-way-to-implement-moving-window-in-python-for-loop"
    def __init__(self, tokens, window_size, step):
        self.current = -step
        self.last = len(tokens.input_ids) - window_size + 1
        self.remaining = (len(tokens.input_ids) - window_size) % step
        self.tokens = tokens
        self.window_size = window_size
        self.step = step

    def __iter__(self):
        return self

    def __next__(self):
        self.current += self.step
        if self.current < self.last:
            return LMInput(input_ids=self.tokens.input_ids[self.current : self.current + self.window_size],
                           attention_mask=self.tokens.attention_mask[self.current : self.current + self.window_size])
        elif self.remaining:
            self.remaining = 0
            return LMInput(input_ids=self.tokens.input_ids[-self.window_size:],
                           attention_mask=self.tokens.attention_mask[-self.window_size:])
        else:
            raise StopIteration
        
def sliding_window(sample):
    samples = {"input_ids": [], "attention_mask": [], "id":[]}
    
    #print(sample.keys())
    
    for i in range(len(sample["id"])):
        for j, s_sample in enumerate(MovingWindow(tokenizer(sample["text"][i]), 2048, 1024)):
            samples["input_ids"].append(s_sample.input_ids)
            samples["attention_mask"].append(s_sample.attention_mask)
            _id = sample["id"][i]
            samples["id"].append(f"{_id}_{j}")
            #samples["text"].append(sample["text"][i])

    #print(samples)
    return samples
        


In [19]:
dataset = dataset.map(sliding_window, batched=True, batch_size=8, remove_columns=["text"])

Map: 100%|██████████| 1052/1052 [00:23<00:00, 45.52 examples/s]


In [23]:
#dataset[0]["input_ids"]

In [33]:
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union


@dataclass
class ConvertToTensor:

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        
        samples = {"input_ids": [], "attention_mask": [], "id":[]}
        
        for feature in features:
            samples["id"].append(feature["id"])
            samples["input_ids"].append(feature["input_ids"])
            samples["attention_mask"].append(feature["attention_mask"])
        
        samples["input_ids"] = torch.as_tensor(samples["input_ids"])
        samples["attention_mask"] = torch.as_tensor(samples["attention_mask"])
        
        return samples


dl = torch.utils.data.DataLoader(dataset,
                                         batch_size=1, 
                                         collate_fn=ConvertToTensor(),
                                         pin_memory=True)

In [34]:
context_tokens = 68

for b_sample in dl:
    
    with torch.no_grad():
       
        b_id = b_sample.pop("id")
        logits = model(**b_sample).logits[:,context_tokens:-1,:] # skip last
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
            
        target_ids = b_sample.input_ids[:,context_tokens+1:, None].long() # skip first + context  
        log_target_probs = torch.gather(log_probs, -1, target_ids).squeeze(-1)
        
    break

<class 'dict'>


AttributeError: 'dict' object has no attribute 'input_ids'

In [26]:
b_sample

{'input_ids': [tensor([[10248,  6180,    13,  ...,  7030,   340,    13]])],
 'attention_mask': [tensor([[1, 1, 1,  ..., 1, 1, 1]])]}

In [ ]:
estimator = HFNIEstimator("EleutherAI/gpt-neo-125m")

In [5]:
#for r in estimator.ni_from_generator(read_jsonl("../../clean_datasets/math__openai_grade_school_clean.jsonl")):
#    print(r)

  0%|          | 3/8792 [00:00<27:50,  5.26it/s]  

{'id': tensor(0), 'information': tensor(279.0507), 'seq_len': 83}
{'id': tensor(1), 'information': tensor(288.3089), 'seq_len': 81}
{'id': tensor(2), 'information': tensor(414.1760), 'seq_len': 134}
{'id': tensor(3), 'information': tensor(439.0298), 'seq_len': 151}


  0%|          | 7/8792 [00:00<13:38, 10.74it/s]

{'id': tensor(4), 'information': tensor(288.2872), 'seq_len': 91}
{'id': tensor(5), 'information': tensor(513.2334), 'seq_len': 189}
{'id': tensor(6), 'information': tensor(313.2731), 'seq_len': 119}
{'id': tensor(7), 'information': tensor(518.6471), 'seq_len': 214}


  0%|          | 11/8792 [00:01<11:52, 12.33it/s]

...

  1%|          | 85/8792 [00:04<06:53, 21.03it/s]

{'id': tensor(81), 'information': tensor(526.5134), 'seq_len': 231}
{'id': tensor(82), 'information': tensor(433.8779), 'seq_len': 132}
{'id': tensor(83), 'information': tensor(558.9106), 'seq_len': 210}
{'id': tensor(84), 'information': tensor(408.6118), 'seq_len': 111}
{'id': tensor(85), 'information': tensor(341.5211), 'seq_len': 109}


  1%|          | 90/8792 [00:05<08:10, 17.76it/s]

{'id': tensor(86), 'information': tensor(306.8661), 'seq_len': 93}
{'id': tensor(87), 'information': tensor(390.4664), 'seq_len': 110}
{'id': tensor(88), 'information': tensor(283.0951), 'seq_len': 77}
{'id': tensor(89), 'information': tensor(499.7301), 'seq_len': 158}


KeyboardInterrupt: 

In [81]:
{v:k for k,v in tokenizer.vocab.items()}[262]

'Ġthe'